#Mathematical Background

We assume an infinite acceptor $\mathcal{A} = (\Sigma, \  \mathcal{S}, \  s_0 \in \mathcal{S}, \  \mathcal{F} \subset \mathcal{S}, \  \delta: \mathcal{S} \times \Sigma \rightarrow \mathcal{S})$, that implements a tic-tac-toe game on infinite board where:

* $\Sigma$ - input alphabet, consist of tuples $\langle \mathcal{i}, \mathcal{j}\rangle \quad i, j = 0..8$

* $\mathcal{S}$ - infinite set of states that consist of all combinations of set of players: $\ \mathcal{P} = \ \{"X", "O"\}$, and set of endless board states. Let $\mathcal{B}$ be the infinite set of all endless board states then it has a structure like $\mathcal{N} \times \mathcal{N} \times \mathcal{C}$, where $\mathcal{C} = \{"X", "O", "*"\}$  

* $s_0 \in \mathcal{S}$ - is initial state has player $"X"$ and $"*"$ on all coordinates.

* $\mathcal{F} \subset \mathcal{S}$ - set of final states, consists of states in which there is a winning combination on the board: five in a row (horizontal, vertical and diagonal) and draw combinations

* $\delta: \mathcal{S} \times \Sigma \rightarrow \mathcal{S}$ - transition fucntion: that accept coordinates and put corresponding symbol into this coordinate


In [ ]:
from prettytable import PrettyTable
from typing import List, Tuple


class Board:
    """Implements a board state. Only magical methods are implemented
    to interact like with lists but with pretty output
    """
    def __init__(self, board: List[List[str]]) -> None:
        self.__board = board

    def __getitem__(self, item: int):
        return self.__board[item]

    def __setitem__(self, key, value: str) -> None:
        self.__board[key] = value

    def __eq__(self, other):
        return self.__board == other.__board

    def __str__(self) -> str:
        table_head = [""] + [i for i in range(9)]
        table = PrettyTable(table_head)
        for i, row in enumerate(self.__board):
            table.add_row([i] + row)

        return str(table)


class State:

    def __init__(self, player: str, board: Board) -> None:
        self.player: str = player
        self.board: Board = board


class TicTacToe:

    def __init__(self):
        self.__current_board_state = State("X", Board([["-"] * 9 for i in range(9)]))
        self.__input_alphabet = set((i, j) for i in range(8) for j in range(8))
        self.__filled_cells = dict() # cell on board that filled with "X" or "O"
        self.__current_focus = (0, 0)

    def next_state(self, coords: Tuple[int, int]):

        if coords not in self.__input_alphabet:
            return "Oops"

        elif self.__current_board_state.board[coords[0]][coords[1]] != "-":
            return "Oops"

        else:
            self.focus(coords)
            self.__filled_cells[self.__current_focus] = self.__current_board_state.player
            self.__current_board_state.player = "X" if self.__current_board_state == "X" else "O"
            self.__build_board()
            return self.__current_board_state.board

    def is_final_state(self):
        vertical = "".join(self.__current_board_state.board[4])
        horizontal = "".join([row[0] for row in self.__current_board_state.board])
        main_diagonal = "".join([row[i] for i, row in enumerate(self.__current_board_state.board)])
        side_diagonal = "".join([row[-i - 1] for i, row in enumerate(self.__current_board_state.board)])
        xs_wins = "XXXXX"
        os_wins = "OOOOO"
        if xs_wins in horizontal or os_wins in horizontal:
            return True
        elif xs_wins in vertical or os_wins in vertical:
            return True
        elif xs_wins in main_diagonal or os_wins in main_diagonal:
            return True
        elif xs_wins in side_diagonal or os_wins in side_diagonal:
            return True
        return False

    def __build_board(self):
        board = [["-"] * 9 for i in range(9)]
        for fst_coord, row in enumerate(board):
            for snd_coord, column in enumerate(board):
                coords = (self.__current_focus[0] + (fst_coord - 4),
                           self.__current_focus[1] - (snd_coord - 4))
                if coords in self.__filled_cells:
                    board[fst_coord][snd_coord] = self.__filled_cells[coords]
        self.__current_board_state.board = Board(board)

    def focus(self, coords: Tuple[int, int]) -> None:
        self.__current_focus = (self.__current_focus[0] + (coords[0] - 4),
                                self.__current_focus[1] - (coords[1] - 4))
        self.__build_board()

    def cur_board(self):
        return self.__current_board_state.board


a = TicTacToe()

board_state = Board([["-"] * 9 for i in range(9)])
print(board_state)

while not a.is_final_state():
    command, *coords = input("Enter command(focus, grab):").split()
    coords = tuple(int(i) for i in coords)
    if command == "focus":
        a.focus(coords)
        b = a.cur_board()
        print(b)

    if command == "grab":
        b = a.next_state(coords)
        print(b)
    else:
        print("try again")


